# `Практикум по программированию на языке Python`
<br>

## `Занятие 2: Функции, модули, классы. Итераторы и генераторы`
<br><br>

### `Роман Ищенко (sir.rois@yandex.ru)`

#### `Москва, 2022`

### `Организация кода`

Функции, модули, классы позволяют организовать и структурировать код, избежать повторяющихся фрагментов.

Кроме того:
- упростить отладку
- упростить создание тестов
- улучшить «понимаемость» кода

### `Функции range и enumerate`

In [1]:
r = range(2, 10, 3)
print(type(r))

for e in r:
    print(e, end=' ')

<class 'range'>
2 5 8 

In [5]:
for index, element in enumerate(list('abcdef')):
    print(index, element, end='   ')

0 a   1 b   2 c   3 d   4 e   5 f   

### `Функция zip`

In [7]:
z = zip([1, 2, 3], 'abc')
print(type(z))

for a, b in z:
    print(a, b, end='  ')

<class 'zip'>
1 a  2 b  

In [21]:
for e in zip('abcdef', 'abc'):
    print(e)

('a', 'a')
('b', 'b')
('c', 'c')


In [9]:
for a, b, c, d in zip('abc', [1,2,3], [True, False, None], 'xyz'):
    print(a, b, c, d)

a 1 True x
b 2 False y
c 3 None z


### `Определение собственных функций`

In [14]:
def function(arg_1, arg_2=None):
    """Factorial function"""
    print(arg_1, arg_2)

function(10)
function(10, 20)

10 None
10 20


Функция - это тоже объект, её имя - просто символическая ссылка:

In [15]:
f = function
f(10)

print(function is f)

10 None
True


### `Определение собственных функций`

У функции есть атрибуты, например: `__doc__` или `__dict__` (список атрибутов функции)

In [18]:
print(dir(f)[: 4], len(dir(f)))
f.__doc__

['__annotations__', '__call__', '__class__', '__closure__'] 35


'Factorial function'

In [20]:
# del f.my_attr
print(f.__dict__)
f.my_attr = 0
print(f.__dict__)

{}
{'my_attr': 0}


### `Определение собственных функций`

In [30]:
retval = f(10)
print(retval)

10 None
None


In [13]:
def factorial(n):
    return n * factorial(n - 1) if n > 1 else 1  # recursion

print(factorial(1))
print(factorial(2))
print(factorial(4))


1
2
24


### `Глубина рекурсии`

При достижении глубины вызовов, заданной в системе, программа выбросит исключение `RecursionError`

Заданную глубину рекурсии можно узнать и изменить, если есть такая необходимость.

In [11]:
import sys
print(sys.getrecursionlimit())

sys.setrecursionlimit(1000)
print(sys.getrecursionlimit())


3000
1000


### `Передача аргументов в функцию`

Параметры в Python всегда передаются по ссылке

In [33]:
def function(scalar, lst):
    scalar += 10
    print(f'Scalar in function: {scalar}')

    lst.append(None)
    print(f'List in function: {lst}')

In [34]:
s, l = 5, []
function(s, l)

print(s, l)

Scalar in function: 15
Scalar in function: [None]
5 [None]


### `Передача аргументов в функцию`

Использовать изменяемые объекты в качестве значений по умолчанию опасно и не нужно

In [24]:
def func_1(list_of_items, my_set=set()):
    final_list = list()
    for item in list_of_items:
        if item not in my_set:
            my_set.add(item)
            final_list.append(item)
    return final_list

my_list = [1, 2, 3]
print(func_1(my_list))
print(func_1(my_list))
my_list

[1, 2, 3]
[]


[1, 2, 3]

### `Передача аргументов в функцию`

In [26]:
def f(a, *args):
    print(type(args))
    print([v for v in [a] + list(args)])
    
f(10, 2, 6, 8)

<class 'tuple'>
[2, 6, 8]


In [27]:
def f(*args, a):
    print([v for v in [a] + list(args)])
    print()

f(2, 6, 8, a=10)
f(2, 3, 4)

[10, 2, 6, 8]



TypeError: f() missing 1 required keyword-only argument: 'a'

In [227]:
def f(a, *args, **kw):
    print(type(kw))
    print([v for v in [a] + list(args) + [(k, v) for k, v in kw.items()]])

f(2, *(6, 8), **{'arg1': 1, 'arg2': 2})

<class 'dict'>
[2, 6, 8, ('arg1', 1), ('arg2', 2)]


In [41]:
def test(a, *args, b, c=100, **kw):
    print([v for v in [a] + list(args) + [b] + [c] + [(k, v) for k, v in kw.items()]])

test(1, 2, 3, b=10, d=1000)

[1, 2, 3, 10, 100, ('d', 1000)]


### `Спецификация позиционности параметров`

- Начиная с версии 3.8 в Python появилась возможность явно запрещать передачу параметров по имени или позиции
- Мотивация - дать возможность запрета именованных параметров там, где это нужно (это позволяет эмулировать C-подобные функции)
- Часть встроенных функций самого языка и раньше имели такое свойство, например, `math.exp`:

In [14]:
import math
#help(math.exp) -> ... exp(x, /) ...

In [13]:
print(math.exp(5))
#print(math.exp(x=5)) -> TypeError: math.exp() takes no keyword arguments

148.4131591025766


Синтаксис заголовка функции:

In [16]:
def func(positional_only_parameters, /, positional_or_keyword_parameters, *, keyword_only_parameters): pass

In [23]:
def f(a, b, /, c, *, d): pass
#f(1, 2, 3, 4) -> TypeError: func() takes 3 positional arguments but 4 were given
#f(a=1, b=2, c=3, d=4) -> TypeError: func() got some positional-only arguments passed as keyword arguments: 'a, b'
f(1, 2, 4, d=3) # OK
f(1, 2, d=4, c=3) # OK

### `Области видимости переменных`

В Python есть 4 основных уровня видимости:

- Встроенная (buildins) - на этом уровне находятся все встроенные объекты (функции, классы исключений и т.п.)<br><br>
- Глобальная в рамках модуля (global) - всё, что определяется в коде модуля на верхнем уровне<br><br>
- Объемлющей функции (enclosed) - всё, что определено в функции верхнего уровня<br><br>
- Локальной функции (local) - всё, что определено в функции нижнего уровня

<br><br>
Есть ещё области видимости переменных циклов, списковых включений и т.п.

### `Правило разрешения области видимости LEGB при чтении`

In [38]:
def outer_func(x):
    def inner_func(x):
        return len(x)
    return inner_func(x)

In [39]:
print(outer_func([1, 2]))

2


Кто определил имя `len`?

- на уровне вложенной функции такого имени нет, смотрим выше
- на уровне объемлющей функции такого имени нет, смотрим выше
- на уровне модуля такого имени нет, смотрим выше
- на уровне builtins такое имя есть, используем его

### `На builtins можно посмотреть`

In [43]:
import builtins

counter = 0
lst = []
for name in dir(builtins):
    if name[0].islower():
        lst.append(name)
        counter += 1
    
    if counter == 5:
        break

lst

['abs', 'all', 'any', 'ascii', 'bin']

Кстати, то же самое можно сделать более pythonic кодом:

In [45]:
list(filter(lambda x: x[0].islower(), dir(builtins)))[: 5]

['abs', 'all', 'any', 'ascii', 'bin']

### `Локальные и глобальные переменные`

In [47]:
x = 2
def func():
    print('Inside: ', x)  # read
    
func()
print('Outside: ', x)

Inside:  2
Outside:  2


In [ ]:
x = 2
def func():
    x += 1  # write
    print('Inside: ', x)
    
func()  # UnboundLocalError: local variable 'x' referenced before assignment
print('Outside: ', x)

In [50]:
x = 2
def func():
    x = 3
    x += 1
    print('Inside: ', x)
    
func()
print('Outside: ', x)

Inside:  4
Outside:  2


### `Ключевое слово global`

In [52]:
x = 2
def func():
    global x
    x += 1  # write
    print('Inside: ', x)
    
func()
print('Outside: ', x)

Inside:  3
Outside:  3


In [55]:
x = 2
def func(x):
    x += 1
    print('Inside: ', x)
    return x
    
x = func(x)
print('Outside: ', x)

Inside:  3
Outside:  3


### `Ключевое слово nonlocal`

In [46]:
a = 0
def out_func():
    b = 10
    def mid_func():
        c = 20
        def in_func():
            global a
            a += 100
            
            nonlocal c
            c += 100
            
            nonlocal b
            b += 100

            print(a, b, c)
            
        in_func()
    mid_func()

out_func()

100 110 120


__Главный вывод:__ не надо злоупотреблять побочными эффектами при работе с переменными верхних уровней

### `Пример вложенных функций: замыкания`

- В большинстве случаев вложенные функции не нужны, плоская иерархия будет и проще, и понятнее
- Одно из исключений - фабричные функции (замыкания)

In [47]:
def function_creator(n):
    def function(x):
        return x ** n

    return function

f5 = function_creator(5)
f5(2)

32

Объект-функция, на который ссылается `f`, хранит в себе значение `n`

### `Анонимные функции`

- `def` - не единственный способ объявления функции
- `lambda` создаёт анонимную (lambda) функцию


Такие функции часто используются там, где синтаксически нельзя записать определение через `def`

In [88]:
def func(x): return x ** 2
func(6)

36

In [94]:
lambda_func = lambda x: x ** 2  # should be an expression
lambda_func(6)

36

In [91]:
def func(x): print(x)
func(6)

6


In [48]:
lambda_func = lambda x: print(x ** 2)  # as print is function in Python 3.*
lambda_func(6)

36


### `Встроенная функция sorted`

In [52]:
lst = [5, 2, 7, -9, -1]

print(sorted(lst))

[-9, -1, 2, 5, 7]


In [51]:
def abs_comparator(x):
    return abs(x)

print(sorted(lst, key=abs_comparator))

[-1, 2, 5, 7, -9]


In [55]:
print(sorted(lst, key=lambda x: abs(x)))
print(sorted(lst, key=abs))

[-1, 2, 5, 7, -9]
[-1, 2, 5, 7, -9]


### `Встроенная функция sorted`

Более сложный пример

In [56]:
lst = [(1, 0), (1, 1), (1, 2), (0, 3), (2, 1)]
print(sorted(lst))

[(0, 3), (1, 0), (1, 1), (1, 2), (2, 1)]


In [60]:
print(sorted(lst, key=lambda x: x[1])) # сортировка по второму элементу
print(sorted(lst, key=lambda x: x[0] * x[1], reverse=True)) # сортировка по убыванию произведения

[(1, 0), (1, 1), (2, 1), (1, 2), (0, 3)]
[(1, 2), (2, 1), (1, 1), (1, 0), (0, 3)]



### `Встроенная функция filter`

In [220]:
lst = [5, 2, 7, -9, -1]

In [221]:
f = filter(lambda x: x < 0, lst)  # True condition
type(f)  # iterator

filter

In [222]:
list(f)

[-9, -1]

### `Встроенная функция map`

In [123]:
lst = [(1, 0), (1, 1), (1, 2), (0, 3), (2, 1)]

In [61]:
m = map(lambda x: x[0] * x [1], lst)
type(m)  # iterator

map

In [62]:
list(m)

[0, 1, 2, 0, 2]

### `Ещё раз сравним два подхода`

Напишем функцию скалярного произведения в императивном и функциональном стилях:

In [63]:
def dot_product_imp(v, w):
    result = 0
    for i in range(len(v)):
        result += v[i] * w[i]
    return result

In [64]:
dot_product_func = lambda v, w: sum(map(lambda x: x[0] * x[1], zip(v, w)))

In [65]:
print(dot_product_imp([1, 2, 3], [4, 5, 6]))
print(dot_product_func([1, 2, 3], [4, 5, 6]))

32
32


### `Функция reduce`

`functools` - стандартный модуль с другими функциями высшего порядка.

Рассмотрим пока только функцию `reduce`:

In [155]:
from functools import reduce

lst = list(range(1, 10))

reduce(lambda x, y: x * y, lst)

362880

### `Итераторы`

- __Итерабельный объект (iterable)__ - источник данных для итерирования
- __Итератор (iterator)__ - объект-абстракция, извлекающий из источника элемент за элементом и знающий только о том объекте, на котором он в текущий момент остановился


- Итераторы используются для итерирования по последовательностям, при этом:
    - последовательности могут быть неиндексируемыми (например, `set`)
    - в процессе работы элементы могут фильтроваться или преобразовываться
    - итераторы работают лениво


- В Python для итераторов есть две встроенные функции:
    - `iter(x: iterable)` - создаёт итератор для переданной последовательности
    - `next(it: iterator)` - выполняет шаг итерации для переданного итератора

### `Итераторы`

In [54]:
r = iter([1, 2, 3])

print(next(r), next(r), next(r))
#next(r) -> StopIteration

print(type(r))

1 2 3
<class 'list_iterator'>


Итераторы можно получать вызовами стандартных функций:

In [66]:
r = enumerate([1, 2, 3])
print(type(r))

print(next(r), next(r), next(r))
#next(r) -> StopIteration

<class 'enumerate'>
(0, 1) (1, 2) (2, 3)


### `Итераторы и цикл for`

Цикл `for` использует итераторы вместо индексов. Как выглядит `for` снаружи:

In [41]:
for e in {1, 2, 3}:
    print(e, end=' ')

1 2 3 

Как он работает на самом деле:

In [43]:
iterator = iter({1, 2, 3})
while True:
    try:
        i = next(iterator)
        print(i, end=' ')
    except StopIteration:
        break

1 2 3 

### `Итераторы`

Примеры встроенных функций, возвращающих итераторы:

- `enumerate`
- `zip`
- `open`
- `reversed`
- `map` (похож на генератор, но метод `send` не реализует)
- `filter`

Вызов `iter` от итератора вернет тот же самый итератор:

In [48]:
i = iter([1, 2, 3])

print(next(i), next(iter(i)), next(i))

1 2 3


### `Генераторы`

- __Генератор__ - подтип итератора
- В генераторе есть внутреннее изменяемое состояние в виде локальных переменных, которое он хранит автоматически
- В генератор можно посылать данные между итерациями (метод `send`)
- Генераторы можно создавать с помощью генераторных выражений или описывать функциями, в которых `return` заменяется на `yield`

Пример генератора, полученного с помощью выражения:

In [55]:
gen = (x**2 for x in range(5))

print(next(gen), next(gen))
print(type(gen))

0 1
<class 'generator'>


### `Ключевое слово yield`

- `yield` - это слово, по смыслу похожее на `return`<br><br>
- Но используется в функциях, возвращающих генераторы<br><br>
- При вызове такой функции тело не выполняется, функция только возвращает генератор<br><br>
- В первых запуск функция будет выполняться от начала и до `yield`<br><br>
- После выхода состояние функции сохраняется<br><br>
- На следующий вызов будет проводиться итерация цикла и возвращаться следующее значение<br><br>
- И так далее, пока не кончится цикл каждого `yield` в теле функции<br><br>
- После этого генератор станет пустым

### `Пример функции-генератора`

In [56]:
def my_range(n):
    yield 'You really want to run this generator?'

    i = -1
    while i < n:
        i += 1
        yield i

In [57]:
gen = my_range(3)
while True:
    try:
        print(next(gen), end='   ')
    except StopIteration:  # we want to catch this type of exceptions
        break

You really want to run this generator?   0   1   2   3   

In [58]:
for e in my_range(3):
    print(e, end='   ')

You really want to run this generator?   0   1   2   3   

### `Генераторы и метод send`

- Выражение `yield x` выполняет две вещи:
    1. Возвращает вызывающему генератор коду значение `x`
    2. Возвращает на уровне кода генератора значение `None`


- Расширенный протокол генератора содержит метод `send`, получающий на вход произвольный объект `y`
- В случае вызова `send(y)` выполняется то же, что и при `next`, но `yield` вернет на уровне кода генератора не `None`, а `y`

In [104]:
def create_gen():
    for x in range(5):
        y = yield x
        print(f'y = {y}', end='   ')

In [105]:
gen = create_gen()

print(next(gen), end=' | ')
print(next(gen), end=' | ')

print(gen.send(10), end=' | ')
print(next(gen), end=' | ')

0 | y = None   1 | y = 10   2 | y = None   3 | 

### `Итераторы и функция range`

Результат работы `range` не является итератором, хотя и выполняет его роль:

In [106]:
print('__next__' in dir(zip([], [])))
print('__next__' in dir(range(3)))

True
False


__Особенности объектов__ `range`:
- как и итераторы, выполняются лениво
- являются неизменяемыми (могут быть ключами словаря)
- имеют полезные атрибуты (`len`, `index`, `__getitem__`)
- по ним можно итерироваться многократно

### `Модуль itetools`

- Модуль представляет собой набор инструментов для работы с итераторами и последовательностями<br><br>
- Содержит три основных типа итераторов:<br><br>
    - бесконечные итераторы
    - конечные итераторы
    - комбинаторные итераторы<br><br>

- Позволяет эффективно решать небольшие задачи вида:<br><br>
    - итерирование по бесконечному потоку
    - слияние в один список вложенных списков
    - генерация комбинаторного перебора сочетаний элементов последовательности
    - аккумуляция и агрегация данных внутри последовательности

### `Модуль itetools: примеры`

In [203]:
from itertools import count

for i in count(start=0):
    print(i, end='  ')
    if i == 5:
        break

0  1  2  3  4  5  

In [205]:
from itertools import cycle
 
count = 0
for item in cycle('XYZ'):
    if count > 4:
        break
    print(item, end='  ')
    count += 1

X  Y  Z  X  Y  

### `Модуль itetools: примеры`

In [67]:
from itertools import accumulate

for i in accumulate(range(1, 6), lambda x, y: x * y):
    print(i, end='  ')

1  2  6  24  120  

In [108]:
from itertools import chain

for i in chain([1, 2], [3], [4]):
    print(i, end='  ')

1  2  3  4  

### `Модуль itetools: примеры`

In [215]:
from itertools import groupby
 
vehicles = [('Ford', 'Taurus'), ('Dodge', 'Durango'),
            ('Chevrolet', 'Cobalt'), ('Ford', 'F150'),
            ('Dodge', 'Charger'), ('Ford', 'GT')]
 
sorted_vehicles = sorted(vehicles)
 
for key, group in groupby(sorted_vehicles, lambda x: x[0]):
    for maker, model in group:
        print('{model} is made by {maker}'.format(model=model, maker=maker))
    
    print ("**** END OF THE GROUP ***\n")

Cobalt is made by Chevrolet
**** END OF THE GROUP ***

Charger is made by Dodge
Durango is made by Dodge
**** END OF THE GROUP ***

F150 is made by Ford
GT is made by Ford
Taurus is made by Ford
**** END OF THE GROUP ***



### `Модуль itetools: примеры`

In [76]:
from itertools import product, permutations, combinations
print (list(product("AB", repeat=2)))
print (list(permutations("ABC")))
print (list(combinations("ABC", 2)))


[('A', 'A'), ('A', 'B'), ('B', 'A'), ('B', 'B')]
[('A', 'B', 'C'), ('A', 'C', 'B'), ('B', 'A', 'C'), ('B', 'C', 'A'), ('C', 'A', 'B'), ('C', 'B', 'A')]
[('A', 'B'), ('A', 'C'), ('B', 'C')]


### `Модули`

Любой файл с расширением `.py` является модулем

Оператор `import` импортирует модуль и создаёт ссылку на модуль в текущей области видимости:
- ищет в sys.path имя
- компилирует в байт-код (если версия уже неактуальна)
- выполняет код (только первый раз)

In [ ]:
import itertools

import itertools as it

from itertools import product as prd

# from itertools import *

Так как модули тоже объекты, у них тоже есть атрибуты

In [79]:
import itertools
print(itertools.__name__)

# print(my_module.__path__)

itertools


### `Модули`

У модулей, переданных на выполнение интерпретатору, `__name__ == '__main__'`

In [ ]:
if __name__ == '__main__':
    # Код, выполняющийся только если именно этот модуль передан интерпретатору
    pass

In [80]:
# Если модуль нужно перезагрузить принудительно
from importlib import reload
reload(itertools)


<module 'itertools' (built-in)>

### `Классы`

Класс объявлется с помощью ключевого слова `class`

In [1]:
class MyClass:
    def __init__(self, value):
        self.value = value

    def get_value(self):
        return self.value

my_obj = MyClass(2)
print(my_obj.get_value())

2


Каждый объект экземпляра наследует атрибуты класса и  приобретает свое собственное пространство имен

### `Функция __init__`

- Главное: `__init__` - не конструктор! Она ничего не создаёт и не возвращает

- Созданием объекта занимается функция `__new__`, переопределять которую без необходимости не надо

- `__init__` получает на вход готовый объект и инициализирует его атрибуты

В отличие от C++, атрибуты можно добавлять/удалять на ходу:

In [2]:
class Cls:
    pass

cls = Cls()
cls.field = 'field'
print(cls.field)

del cls.field
print(cls.field)  # AttributeError: 'Cls' object has no attribute 'field'

field


### `Параметр self`

- Метод класса отличается от обычной функции только наличием объекта `self` в качестве первого аргумента

- Это то же самое, что происходит в C++/Java (там аналогом `self` является указатель/ссылка `this`)

- Название `self` является общим соглашением, но можно использовать и другое (не надо!)

- Метод класса, не получающий на вход `self` является _статическим_

- Статические методы часто используются для специализированного создания объектов класса


### `Доступ к атрибутам`

- Для работы с атрибутами есть функции `getattr`, `setattr` и `delattr`
- Их основное преимущество - оперирование именами атрибутов в виде строк

In [3]:
cls = Cls()

setattr(cls, 'some_attr', 'some')

print(getattr(cls, 'some_attr'))

delattr(cls, 'some_attr')

print(getattr(cls, 'some_attr'))

some


AttributeError: 'Cls' object has no attribute 'some_attr'

### `Class magic methods (магические методы)`

- Магические методы придают объекту класса определённые свойства

- Такие методы получают `self` вызываются интерпретатором неявно

- Например, операторы — это магические методы

Рассмотрим несколько примеров:

In [6]:
class Cls:
    def __init__(self):  # initialize object
        self.name = 'Some class'
    
    def __repr__(self):  # str for printing object
        return 'Class: {}'.format(self.name)
    
    def __call__(self, counter):  # call == operator() in C++
        return self.name * counter

cls = Cls()
print(cls)
print(cls(2))

Class: Some class
Some classSome class


### `Class magic methods`


Ещё примеры магических методов:

In [ ]:
class Cls():
    def __add__(self, other): pass # x + other

    def __sub__(self, other): pass # x - other

    def __mul__(self, value): pass # x * value

    def __rmul__(self, value): pass # value * x

    def __int__(self): pass # int(x)

    def __getitem__(self, index): pass # x[index]

    def __setitem__(self, index, value): pass # x[index] = value

### `Ещё об атрибутах`

Протокол свойств позволяет направлять операции чтения, записи и удаления для отдельных атрибутов отдельным функциям и методам

`attribute = property(fget, fset, fdel, doc)`

In [8]:
class Cls():
    def __init__(self):
        self._value = 0

    def getValue(self):
        print(f'Getting value')
        return self._value

    def setValue(self, val):
        print(f'Setting value to {val}')
        self._value = val

    value = property(getValue, setValue, None, 'VALUE')


cls = Cls()

print(cls.value)

cls.value = 10
print(cls.value)


Getting value
0
Setting value to 10
Getting value
10


### `Итератор`

Класс, который должен поддерживать магические методы `__iter__` и `__next__`

In [7]:
class iter_example:

    def __init__(self, cnt):
        self.max = cnt

    def __iter__(self):
        self.curr = 0
        return self

    def __next__(self):
        if self.curr < self.max:
            self.curr += 1
            return self.curr ** 2

        raise StopIteration


a = iter(iter_example(5))
print(*list(a))


1 4 9 16 25


## `Спасибо за внимание!`